In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
import pickle
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)
from collections import defaultdict
from itertools import islice, combinations
from datetime import datetime as dt
import warnings
warnings.filterwarnings("ignore")
# import seaborn as sns
# import matplotlib.pyplot as plt

# from ctgan import CTGANSynthesizer
# # 폰트 적용
plt.rcParams['font.family'] = 'Malgun Gothic'


In [2]:
# Compare Algorithms
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.utils import resample
import random
import time
import lightgbm as lgb
import xgboost as xgb
import catboost as cab
import string
import csv

In [3]:
import os
print(os.getcwd())

C:\Users\kcs\df_syn_generation_m


In [4]:
os.chdir("C:/Users/kcs/df_syn_generation_m")

## 1. 데이터프레임 불러오기

In [5]:
X_org_train = pd.read_csv('./label_org/encoding_train.csv', encoding='utf-8-sig')
y_org_train = pd.read_csv('./label_org/y_train.csv', encoding='utf-8-sig')
X_org_test = pd.read_csv('./label_org/encoding_valid.csv', encoding='utf-8-sig')
y_org_test = pd.read_csv('./label_org/y_valid.csv', encoding='utf-8-sig')

In [6]:
X_syn_train = pd.read_csv('./label_syn/encoding_train.csv', encoding='utf-8-sig')
y_syn_train = pd.read_csv('./label_syn/y_train.csv', encoding='utf-8-sig')
X_syn_test = pd.read_csv('./label_syn/encoding_test.csv', encoding='utf-8-sig')
y_syn_test = pd.read_csv('./label_syn/y_test.csv', encoding='utf-8-sig')

## 2. 다중알고리즘 성능비교

In [9]:
clf_list = [LogisticRegression(solver='lbfgs'),
            DecisionTreeClassifier(max_depth=4),
            RandomForestClassifier(n_estimators=100, max_depth=4),  # You can tune some hyper-parameters.
            AdaBoostClassifier(),
            lgb.LGBMClassifier(),
            cab.CatBoostClassifier(depth=4, verbose=False),
            xgb.XGBClassifier(n_estimators=100, max_depth=4,objective= 'binary:logistic', eval_metric='logloss')]

### 2-1. 원본데이터 성능비교

In [10]:

for clf in clf_list:
        
        # Trick to make a unique identifier
        exp_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10)) 

        # substance
        X_train = X_org_train
        y_train = y_org_train
        X_test = X_org_test
        y_test = y_org_test
            
            
        # Check starting time
        start = time.time()
        
        # Train a classifier
        clf = clf.fit(X_train, y_train)
        
        # Predict frauds in test data
        y_pred = clf.predict_proba(X_test)[:, 1]
        
        y_preds = []
        # Append the predictions of each classifier
#         np.append(y_pred,y_preds)
        y_preds.append(y_pred)
        
        # Save the predictions of each classifier in the name of 'exp_id' as .npy files
        np.save(f'./results/intermediary/y_pred_{exp_id}', y_pred)
        np.save(f'./results/intermediary/y_test_{exp_id}', y_test)
        
        # Save all the models as sav files
        filename = f'./results/intermediary/model_{exp_id}.sav'
        pickle.dump(clf, open(filename, 'wb'))
    
        # The reason to choose AUC: 
        # https://datascience.stackexchange.com/questions/806/advantages-of-auc-vs-standard-accuracy
        clas_auc = round(metrics.roc_auc_score(y_test, y_pred), 4)

        # Check the ending time
        done = time.time()
        
        # Check the runnin time
        elapsed = round(done-start, 4)
        
#         # Print the results (content)
#         print(f'{exp_id},{len(X_train)},{len(X_test)},\
#               {str(clf)[:10]},{clas_auc},{elapsed}')
#         print(f'{str(clf)[:10]},  {clas_auc},  {elapsed}')

#         # Print the results (content)
#         print('Classifier','AUC','Time(sec)')
        print(f'{str(clf)[:15]},    {clas_auc},  {elapsed}')
#         print("Classifier: {0:.4f}, AUC: {1:.4f},Time(sec):{2:.4f}\n".format(f'{str(clf)[:10]},{clas_auc}, {elapsed}))


LogisticRegress,    0.542,  0.1775
DecisionTreeCla,    0.6872,  0.0678
RandomForestCla,    0.7018,  1.2396
AdaBoostClassif,    0.6914,  1.125
LGBMClassifier(,    0.7104,  0.1207
<catboost.core.,    0.7002,  3.142
XGBClassifier(b,    0.6831,  0.6682


### 2-2. 가상데이터 성능비교

In [11]:

for clf in clf_list:
        
        # Trick to make a unique identifier
        exp_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10)) 

        # substance
        X_train = X_syn_train
        y_train = y_syn_train
        X_test = X_syn_test
        y_test = y_syn_test
            
            
        # Check starting time
        start = time.time()
        
        # Train a classifier
        clf = clf.fit(X_train, y_train)
        
        # Predict frauds in test data
        y_pred = clf.predict_proba(X_test)[:, 1]
        
        y_preds = []
        # Append the predictions of each classifier
#         np.append(y_pred,y_preds)
        y_preds.append(y_pred)
        
        # Save the predictions of each classifier in the name of 'exp_id' as .npy files
        np.save(f'./results/intermediary1/y_pred_{exp_id}', y_pred)
        np.save(f'./results/intermediary1/y_test_{exp_id}', y_test)
        
        # Save all the models as sav files
        filename = f'./results/intermediary1/model_{exp_id}.sav'
        pickle.dump(clf, open(filename, 'wb'))
    
        # The reason to choose AUC: 
        # https://datascience.stackexchange.com/questions/806/advantages-of-auc-vs-standard-accuracy
        clas_auc = round(metrics.roc_auc_score(y_test, y_pred), 4)

        # Check the ending time
        done = time.time()
        
        # Check the runnin time
        elapsed = round(done-start, 4)
        
#         # Print the results (content)
#         print(f'{exp_id},{len(X_train)},{len(X_test)},\
#               {str(clf)[:10]},{clas_auc},{elapsed}')
#         print(f'{str(clf)[:10]},  {clas_auc},  {elapsed}')

#         # Print the results (content)
#         print('Classifier','AUC','Time(sec)')
        print(f'{str(clf)[:15]},    {clas_auc},  {elapsed}')
#         print("Classifier: {0:.4f}, AUC: {1:.4f},Time(sec):{2:.4f}\n".format(f'{str(clf)[:10]},{clas_auc}, {elapsed}))


LogisticRegress,    0.5254,  0.1805
DecisionTreeCla,    0.6081,  0.0668
RandomForestCla,    0.6351,  1.1342
AdaBoostClassif,    0.6186,  1.058
LGBMClassifier(,    0.7426,  0.1077
<catboost.core.,    0.6912,  3.1281
XGBClassifier(b,    0.7033,  0.5136
